In [18]:
import time, array, random, copy, math
import numpy as np
import pandas as pd
from collections import defaultdict

In [19]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rcParams['text.latex.preamble'] ='\\usepackage{libertine}\n\\usepackage[utf8]{inputenc}'

import seaborn
seaborn.set(style='whitegrid')
seaborn.set_context('notebook')

In [20]:
from deap import algorithms, base, benchmarks, tools, creator

In [21]:
random.seed(a=81)

In [22]:
data = pd.read_excel('VD-LVTN.xlsx', 3, None)

In [23]:
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0,x,x,0,0.058824,3,15,90,4,15,95,x,x,x,x,x,x
1,1,0,FS,0,0.058824,5,135,87,6,135,92,x,x,x,x,x,x
2,2,1,FS,0,0.000000,50,x,x,x,x,x,x,x,x,x,x,x
3,3,0,FS,0,0.058824,7,98,85,9,90,80,10,85,85,x,x,x
4,4,3,FS,0,0.058824,19,612,80,23,550,85,25,581,88,x,x,x
5,5,4,FS,0,0.058824,42,447,85,46,515,82,50,481,85,x,x,x
6,6,5,FS,0,0.058824,7,52,89,9,45,75,10,48.5,78,x,x,x
7,7,"6,2","FS,FS","0,0",0.058824,35,5780,85,40,5690,89,43,5735,92,30,5910,75
8,8,7,FS,0,0.058824,47,1285,95,52,1240,85,55,1262.5,88,x,x,x
9,9,8,FF,0,0.058824,11,212,85,14,200,77,16,206,80,x,x,x


In [24]:
contraints = list(data[[1,2,3]].values)

In [25]:
adj_list = defaultdict(lambda:[])
parent_list = defaultdict(lambda:[])
for idx, (u, v, z) in enumerate(contraints):
    if u == 'x':
        continue
    adj_vert = [int(x) for x in str(u).split(',')]
    adj_constr = str(v).split(',')
    adj_delay = [int(x) for x in str(z).split(',')]
    for i, j, k in zip(adj_vert, adj_constr, adj_delay):
        adj_list[i] += [(idx, j, k)]
        parent_list[idx] += [(i, j, k)]

In [26]:
for i in range(len(contraints)):
    if not adj_list[i]:
        adj_list[i] += [('F', 'FS', 0)]
        parent_list['F'] += [(i, 'FS', 0)]
    if not parent_list[i]:
        adj_list['S'] += [(i, 'FS', 0)]
        parent_list[i] += [('S', 'FS', 0)]

In [27]:
class DiGraph:

    def __init__(self, adj_list: dict, parent_list: dict, num_node: int):
        self.__adj_list = adj_list
        self.__parent_list = parent_list
        self.__cpm = defaultdict(lambda:defaultdict(lambda:0))
        self.__duration = defaultdict(lambda:0)
        self.__n = num_node
        self.__weight = [1./num_node] * num_node
        self.__cost = [0] * num_node
        self.__quality = [0] * num_node

    def reset(self):
        self.__init__(self.__adj_list, self.__parent_list, self.__n)

    def forward_pass(self):
        q = []
        q.append('S')
        while q:
            u = q.pop(0)
            self.__cpm[u]['EF'] = self.__cpm[u]['ES'] + self.__duration[u]
            for v, rela, delay in self.__adj_list[u]:
                if rela == 'FF':
                    self.__cpm[v]['ES'] = max(self.__cpm[v]['ES'], self.__cpm[u]['EF'] - self.__duration[v] + delay)
                elif rela == 'FS':
                    self.__cpm[v]['ES'] = max(self.__cpm[v]['ES'], self.__cpm[u]['EF'] + delay)
                q.append(v)
    
    def get_cpm(self):
        return self.__cpm

    def set_duration(self, duration):
        for i, val in enumerate(duration):
            self.__duration[i] = val if isinstance(val, (int, float)) else 10e3

    def get_total_duration(self):
        return self.__cpm['F']['EF']

    def set_weight(self, weight):
        self.__weight = weight

    def set_cost(self, cost):
        self.__cost = [(x if isinstance(x, (int, float)) else 0) for x in cost]

    def get_total_cost(self):
        return sum(self.__cost)

    def set_quality(self, quality):
        self.__quality = [(x if isinstance(x, (int, float)) else 0) for x in quality]
    
    def get_total_quality(self):
        return sum([(x[0] * x[1]) for x in zip(self.__weight, self.__quality)])

    def get_score(self):
        return self.get_total_duration(), self.get_total_cost(), self.get_total_quality()

In [28]:
a = DiGraph(adj_list, parent_list, len(data[4].values))
a.set_weight(data[4].values)
a.set_duration(data[5].values)
a.set_cost(data[6].values)
a.set_quality(data[7].values)
a.forward_pass()
print(a.get_total_duration())
print(a.get_total_cost())
print(a.get_total_quality())

80000.0
20774
86.70588235294163


In [29]:
candidates = [data[[5,6,7]].values, data[[8,9,10]].values, data[[11,12,13]].values, data[[14,15,16]].values]

In [30]:
N = int(input('Kich thuoc quan the = '))
D = data.shape[0]
MAX_ITER = int(input('So the he toi da = '))
c = float(input('Kha nang tu hoc = '))

In [60]:
a = DiGraph(adj_list, parent_list, D)

def fitness(X):
    a.reset()
    X = [int(round(x)) for x in X]
    X = [candidates[x][i] for i, x in enumerate(X)]
    X = np.transpose(X)
    a.set_duration(X[0])
    a.set_cost(X[1])
    a.set_quality(X[2])
    a.forward_pass()
    return a.get_score()

def fast_non_dominated_sort(values):
    S=[[] for i in range(0,len(values))]
    front = [[]]
    n=[0 for i in range(0,len(values))]
    rank = [0 for i in range(0, len(values))]

    for p in range(0,len(values)):
        S[p]=[]
        n[p]=0
        for q in range(0, len(values)):
            if dominates(values[p], values[q]):
                if q not in S[p]:
                    S[p].append(q)
            elif dominates(values[q], values[p]):
                n[p] = n[p] + 1
        if n[p]==0:
            rank[p] = 0
            if p not in front[0]:
                front[0].append(p)

    i = 0
    while(front[i] != []):
        Q=[]
        for p in front[i]:
            for q in S[p]:
                n[q] =n[q] - 1
                if( n[q]==0):
                    rank[q]=i+1
                    if q not in Q:
                        Q.append(q)
        i = i+1
        front.append(Q)

    del front[len(front)-1]
    return front

def dominates(row, candidateRow):
    return bool(np.any([row[i] > candidateRow[i] for i in range(len(row))]) and np.all([row[i] >= candidateRow[i] for i in range(len(row))]))
    
def crowding_distance(values, front):
    distance = [0 for i in range(0,len(front))]

    sorted1 = sort_by_values(front, values[:, 1])
    sorted2 = sort_by_values(front, values[:, 2])
    sorted3 = sort_by_values(front, values[:, 3])
    distance[0] = 4444444444444444
    distance[len(front) - 1] = 4444444444444444
    for k in range(1,len(front)-1):
        distance[k] = distance[k]+ (values1[sorted1[k+1]] - values2[sorted1[k-1]])/(max(values1)-min(values1))
    for k in range(1,len(front)-1):
        distance[k] = distance[k]+ (values1[sorted2[k+1]] - values2[sorted2[k-1]])/(max(values2)-min(values2))
    for k in range(1,len(front)-1):
        distance[k] = distance[k]+ (values1[sorted2[k+1]] - values2[sorted2[k-1]])/(max(values2)-min(values2))
    return distance

In [61]:
pop = np.random.rand(N,D)*3
pop_fit = np.array([fitness(x) for x in pop])
non_dominated = fast_non_dominated_sort(pop_fit)

In [62]:
non_dominated

[[0, 2, 4, 6, 8, 9], [1, 3, 5], [7]]